# Julia 機器學習：GLM 線性迴歸

## 作業 027：波士頓房價預測資料集

請使用 GLM 中的模型，建立一個預測模型來預測波士頓的房價。

In [4]:
using GLM, RDatasets, MLDataUtils

## 讀取資料

In [5]:
boston = dataset("MASS", "Boston")
first(boston, 10)

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311
8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311
9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311


In [6]:
summary(boston)

"506×14 DataFrame"

In [7]:
# Check for missing values
describe(boston)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Nothing,Nothing,DataType
1,Crim,3.61352,0.00632,0.25651,88.9762,,,Float64
2,Zn,11.3636,0.0,0.0,100.0,,,Float64
3,Indus,11.1368,0.46,9.69,27.74,,,Float64
4,Chas,0.06917,0,0.0,1,,,Int64
5,NOx,0.554695,0.385,0.538,0.871,,,Float64
6,Rm,6.28463,3.561,6.2085,8.78,,,Float64
7,Age,68.5749,2.9,77.5,100.0,,,Float64
8,Dis,3.79504,1.1296,3.20745,12.1265,,,Float64
9,Rad,9.54941,1,5.0,24,,,Int64


In [8]:
using LinearAlgebra

In [9]:
function correlation_matrix(df::DataFrame)::Array{Float64,2}
    reshape([cor(df[:, name1], df[:, name2]) 
         for name1 in names(df) 
             for name2 in names(df)],
        ncol(df), :)
end

boston_corr_mat = correlation_matrix(boston);
UpperTriangular(boston_corr_mat)

14×14 UpperTriangular{Float64,Array{Float64,2}}:
 1.0  -0.200469   0.406583  -0.0558916  …  -0.385064    0.455621   -0.388305
  ⋅    1.0       -0.533828  -0.0426967      0.17552    -0.412995    0.360445
  ⋅     ⋅         1.0        0.062938      -0.356977    0.6038     -0.483725
  ⋅     ⋅          ⋅         1.0            0.0487885  -0.0539293   0.17526 
  ⋅     ⋅          ⋅          ⋅            -0.380051    0.590879   -0.427321
  ⋅     ⋅          ⋅          ⋅         …   0.128069   -0.613808    0.69536 
  ⋅     ⋅          ⋅          ⋅            -0.273534    0.602339   -0.376955
  ⋅     ⋅          ⋅          ⋅             0.291512   -0.496996    0.249929
  ⋅     ⋅          ⋅          ⋅            -0.444413    0.488676   -0.381626
  ⋅     ⋅          ⋅          ⋅            -0.441808    0.543993   -0.468536
  ⋅     ⋅          ⋅          ⋅         …  -0.177383    0.374044   -0.507787
  ⋅     ⋅          ⋅          ⋅             1.0        -0.366087    0.333461
  ⋅     ⋅          ⋅       

In [10]:
hight_corr_ind = (boston_corr_mat[1:end-1, end] .> 0.5) .| (boston_corr_mat[1:end-1, end] .< -0.5) |> findall

3-element Array{Int64,1}:
  6
 11
 13

In [11]:
names(boston)[hight_corr_ind]

3-element Array{String,1}:
 "Rm"     
 "PTRatio"
 "LStat"  

### train, test split

In [13]:
index = MLDataUtils.shuffleobs(collect(1:nrow(boston)));
train_index, test_index = MLDataUtils.splitobs(index);
train, test = boston[train_index, :], boston[test_index, :];

In [14]:
used_formula = @formula(MedV ~ Rm + PTRatio + LStat)
model = GLM.lm(used_formula, train)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

MedV ~ 1 + Rm + PTRatio + LStat

Coefficients:
──────────────────────────────────────────────────────────────────────────
                 Coef.  Std. Error       t  Pr(>|t|)  Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────────────────
(Intercept)  16.6724     4.41607      3.78    0.0002   7.98707   25.3578  
Rm            5.09679    0.479694    10.63    <1e-22   4.15334    6.04023 
PTRatio      -1.04461    0.132475    -7.89    <1e-13  -1.30516   -0.784065
LStat        -0.557403   0.0490047  -11.37    <1e-24  -0.653784  -0.461022
──────────────────────────────────────────────────────────────────────────

In [15]:
rmse(y, y_hat) = sqrt(mean((y-y_hat).^2))
y_hat = predict(model, test)
println("R\u00B2 = $(GLM.r2(model))")
println("RMSE = $(rmse(test.MedV, y_hat))")

R² = 0.718519182513585
RMSE = 5.7915151432873815


### another train set

In [16]:
index = MLDataUtils.shuffleobs(collect(1:nrow(boston)));
train_index, test_index = MLDataUtils.splitobs(index);
train, test = boston[train_index, :], boston[test_index, :];

In [17]:
used_formula = @formula(MedV ~ Rm + PTRatio + LStat)
model = GLM.lm(used_formula, train)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

MedV ~ 1 + Rm + PTRatio + LStat

Coefficients:
──────────────────────────────────────────────────────────────────────────
                 Coef.  Std. Error       t  Pr(>|t|)  Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────────────────
(Intercept)  19.4623     4.81474      4.04    <1e-4    9.9929    28.9318  
Rm            4.49607    0.517118     8.69    <1e-15   3.47902    5.51312 
PTRatio      -0.95839    0.143904    -6.66    <1e-9   -1.24142   -0.675364
LStat        -0.568342   0.0507401  -11.20    <1e-24  -0.668135  -0.468548
──────────────────────────────────────────────────────────────────────────

In [18]:
rmse(y, y_hat) = sqrt(mean((y-y_hat).^2))
y_hat = predict(model, test)
println("R\u00B2 = $(GLM.r2(model))")
println("RMSE = $(rmse(test.MedV, y_hat))")

R² = 0.6748776766130652
RMSE = 4.975325350451077


### prediction

In [21]:
new_X = DataFrame(Rm=[5.3, 4.2, 6.7], PTRatio =[18.5, 17.6, 19.3], LStat=[12.6,14.1,13.2])

,Rm,PTRatio,LStat
,Float64,Float64,Float64
1,5.3,18.5,12.6
2,4.2,17.6,14.1
3,6.7,19.3,13.2


In [22]:
predict(model, new_X)

3-element Array{Union{Missing, Float64},1}:
 18.400229324817715
 13.464586010160488
 23.58701613221472 

In [23]:
GLM.r2(model)

0.6748776766130652

In [24]:
GLM.adjr2(model)

0.6720909138411773